# CRF+LSTM

keras 2.2.4

tensorflow 1.13

pip install git+https://www.github.com/keras-team/keras-contrib.git

In [2]:
import re
import os

In [3]:
char_vocab_path = "CRF/data/char_vocabs.txt" # 字典文件
#train_data_path = 'data/train_data/train_data_000' # 训练数据
#train_data_path = './data/train_data' # 训练数据
#test_data_path = 'data/train_data/train_data_000' # 测试数据

special_words = ['<PAD>', '<UNK>'] # 特殊词表示

# "BIO"标记的标签
#label2idx = {"O": 0,
#             "B-PER": 1, "I-PER": 2,
#             "B-LOC": 3, "I-LOC": 4,
#             "B-ORG": 5, "I-ORG": 6
#            }
label2idx = {'O': 0,
             'B-DISEASE': 1, 'B-DISEASE_GROUP': 2,
             'B-DRUG_DOSAGE': 3, 'B-DRUG_EFFICACY': 4,
             'B-DRUG_INGREDIENT': 5, 'B-DRUG_TASTE': 6,
             'B-FOOD_GROUP':7, 'B-PERSON_GROUP':8,
             'B-SYMPTOM':9, 'B-SYNDROME':10,
             'I-DISEASE': 11, 'I-DISEASE_GROUP': 12,
             'I-DRUG_DOSAGE': 13, 'I-DRUG_EFFICACY': 14,
             'I-DRUG_INGREDIENT': 15, 'I-DRUG_TASTE': 16,
             'I-FOOD_GROUP':17, 'I-PERSON_GROUP':18,
             'I-SYMPTOM':19, 'I-SYNDROME':20
            }

# 索引和BIO标签对应
idx2label = {idx: label for label, idx in label2idx.items()}

# 读取字符词典文件
with open(char_vocab_path, "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
char_vocabs = special_words + char_vocabs

# 字符和索引编号对应
idx2vocab = {idx: char for idx, char in enumerate(char_vocabs)}
vocab2idx = {char: idx for idx, char in idx2vocab.items()}

In [4]:
# 读取训练语料
def read_corpus(corpus_path, vocab2idx, label2idx):
    with open(corpus_path, encoding='utf-8') as fr:
        lines = fr.readlines()

    sent_, tag_ = [], []
    for letter in lines:
        [char,label,_] = re.split('\t|\n',letter)
        sent_.append(char)
        tag_.append(label)

    sent_ids = [vocab2idx[char] if char in vocab2idx else vocab2idx['<UNK>'] for char in sent_]
    tag_ids = [label2idx[label] if label in label2idx else 0 for label in tag_]
    return sent_ids, tag_ids

# 加载训练集
#train_datas, train_labels = read_corpus(train_data_path, vocab2idx, label2idx)
# 加载测试集
#test_datas, test_labels = read_corpus(test_data_path, vocab2idx, label2idx)


In [5]:
train_datas = []
train_labels = []
files = os.listdir('data/train_data')
for file in files:
    train_data_path_i = 'data/train_data/'+file
    train_datas_i, train_labels_i = read_corpus(train_data_path_i, vocab2idx, label2idx)
    train_datas.append(train_datas_i)
    train_labels.append(train_labels_i)
    #if i%10==0:
    #    print(i)

In [6]:
valid_datas = []
valid_labels = []
files = os.listdir('data/valid_data')
for file in files:
    valid_data_path_i = 'data/valid_data/'+file
    valid_datas_i, valid_labels_i = read_corpus(valid_data_path_i, vocab2idx, label2idx)
    valid_datas.append(valid_datas_i)
    valid_labels.append(valid_labels_i)

In [7]:
print(train_datas[50])
print([idx2vocab[idx] for idx in train_datas[50]])
print(train_labels[50])
print([idx2label[idx] for idx in train_labels[50]])

[1, 61, 77, 1, 1, 17, 181, 3093, 3817, 2654, 6214, 1959, 2177, 286, 6802, 5965, 519, 1408, 2644, 2102, 2732, 1842, 889, 2545, 3093, 3817]
['<UNK>', 'b', 'r', '<UNK>', '<UNK>', '3', '、', '治', '疗', '期', '间', '忌', '房', '事', '，', '配', '偶', '如', '有', '感', '染', '应', '同', '时', '治', '疗']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [8]:
import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Masking, Embedding, Bidirectional, LSTM, Dense, Input, TimeDistributed, Activation
from keras.preprocessing import sequence
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras import backend as K
K.clear_session()

EPOCHS = 50
BATCH_SIZE = 128
EMBED_DIM = 32
HIDDEN_SIZE = 16
MAX_LEN = 40
VOCAB_SIZE = len(vocab2idx)
CLASS_NUMS = len(label2idx)
print(VOCAB_SIZE, CLASS_NUMS)

print('padding sequences')
train_datas = sequence.pad_sequences(train_datas, maxlen=MAX_LEN)
train_labels = sequence.pad_sequences(train_labels, maxlen=MAX_LEN)
valid_datas = sequence.pad_sequences(valid_datas, maxlen=MAX_LEN)
valid_labels = sequence.pad_sequences(valid_labels, maxlen=MAX_LEN)
print('x_train shape:', train_datas.shape)
print('x_test shape:', valid_datas.shape)

train_labels = keras.utils.to_categorical(train_labels, CLASS_NUMS)
valid_labels = keras.utils.to_categorical(valid_labels, CLASS_NUMS)
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', valid_labels.shape)

## BiLSTM+CRF模型构建
inputs = Input(shape=(MAX_LEN,), dtype='int32')
x = Masking(mask_value=0)(inputs)
x = Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(x)
x = Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True))(x)
x = TimeDistributed(Dense(CLASS_NUMS))(x)
outputs = CRF(CLASS_NUMS)(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

model.compile(loss=crf_loss, optimizer='adam', metrics=[crf_viterbi_accuracy])
model.fit(train_datas, train_labels, epochs=EPOCHS, verbose=1, validation_split=0.1)

score = model.evaluate(valid_datas, valid_labels, batch_size=BATCH_SIZE)
print(model.metrics_names)
print(score)

Using TensorFlow backend.





6874 21
padding sequences
x_train shape: (6899, 40)
x_test shape: (1987, 40)
trainlabels shape: (6899, 40, 21)
testlabels shape: (1987, 40, 21)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40)                0         
_________________________________________________________________
masking_1 (Masking)          (None, 40)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 40, 32)            219968    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40, 32)            6272      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 40, 21)            693       
_______________________________

6209/6209 [==============================] - 22s 4ms/step - loss: 1.8990 - crf_viterbi_accuracy: 0.9246 - val_loss: 1.9966 - val_crf_viterbi_accuracy: 0.9035
Epoch 23/50
6209/6209 [==============================] - 21s 3ms/step - loss: 1.8956 - crf_viterbi_accuracy: 0.9271 - val_loss: 1.9961 - val_crf_viterbi_accuracy: 0.9022
Epoch 24/50
6209/6209 [==============================] - 22s 4ms/step - loss: 1.8934 - crf_viterbi_accuracy: 0.9289 - val_loss: 1.9987 - val_crf_viterbi_accuracy: 0.8997
Epoch 25/50
6209/6209 [==============================] - 23s 4ms/step - loss: 1.8937 - crf_viterbi_accuracy: 0.9275 - val_loss: 1.9983 - val_crf_viterbi_accuracy: 0.8952
Epoch 26/50
6209/6209 [==============================] - 20s 3ms/step - loss: 1.8916 - crf_viterbi_accuracy: 0.9303 - val_loss: 1.9974 - val_crf_viterbi_accuracy: 0.9018
Epoch 27/50
6209/6209 [==============================] - 18s 3ms/step - loss: 1.8877 - crf_viterbi_accuracy: 0.9324 - val_loss: 1.9962 - val_crf_viterbi_accuracy:

In [9]:
# save model
model.save("model/ch_ner_model.h5")

In [24]:
from keras.models import load_model
import numpy as np

maxlen = 40
sentence = " 北京同仁堂科技发展股份有限公司制药厂  1.忌食辛辣，少进油腻。 2.感冒发热病人不宜服用。 3.有高血压、心脏病、肝病、糖尿病、肾病等慢性病严重者应在医师指导下服用。 4.伴有月经紊乱者，应在医师指导下服用。 5.眩晕症状较重者，应及时去医院就诊。 6.服药2周症状无缓解，应去医院就诊。 7.对本品过敏者禁用，过敏体质者慎用。 8.本品性状发生改变时禁止使用。 9.请将本品放在儿童不能接触的地方。 10.如正在使用其他药品，使用本品前请咨询医师或药师。  本品为浅黄色至棕黄色颗粒，气微香，味微苦。  滋养肝肾、宁心安神。用于更年期综合症属阴虚肝旺症，症见烘热汗出，头晕耳鸣，失眠多梦，五心烦热，腰背酸痛，大便干燥，心烦易怒，舌红少苔，脉弦细或弦细 开水冲服。一次1袋(12g)，一日3次。  如与其他药物同时使用可能会发生药物相互作用，详情请咨询医师或药师。  12g*10袋/盒  用于更年期综合症属阴虚肝旺症  铝塑复合膜包装，每袋装12克，每盒装10袋。  非处方药物（甲类）,中药保护品种二级  12g*10袋/盒  用于更年期综合症属阴虚肝旺更年期综合症气微香，味微苦。"
#sentence = sentence.replace(' ','_')
sentences = sentence.split('。')
y_ner = []

for sent in sentences:
    sent = sent.replace(' ','_')
    sent_chars = list(sent+'。')
    sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]

    sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
    y_pred = model.predict(sent2id_new)
    y_label = np.argmax(y_pred, axis=2)
    y_label = y_label.reshape(1, -1)[0]
    y_ner_ = [idx2label[i] for i in y_label][-len(sent_chars):]
    y_ner.extend(y_ner_)
    #print(idx2label)
    #print(sent_chars)
    #print(sent2id)
    #print(y_ner)

In [25]:
list(zip(list(sentence),y_ner))

[(' ', 'O'),
 ('北', 'O'),
 ('京', 'O'),
 ('同', 'O'),
 ('仁', 'O'),
 ('堂', 'O'),
 ('科', 'O'),
 ('技', 'O'),
 ('发', 'O'),
 ('展', 'O'),
 ('股', 'O'),
 ('份', 'O'),
 ('有', 'O'),
 ('限', 'O'),
 ('公', 'O'),
 ('司', 'O'),
 ('制', 'O'),
 ('药', 'O'),
 ('厂', 'O'),
 (' ', 'O'),
 (' ', 'O'),
 ('1', 'O'),
 ('.', 'O'),
 ('忌', 'O'),
 ('食', 'O'),
 ('辛', 'O'),
 ('辣', 'O'),
 ('，', 'O'),
 ('少', 'O'),
 ('进', 'O'),
 ('油', 'O'),
 ('腻', 'O'),
 ('。', 'O'),
 (' ', 'O'),
 ('2', 'O'),
 ('.', 'O'),
 ('感', 'B-DISEASE'),
 ('冒', 'I-DISEASE'),
 ('发', 'B-SYMPTOM'),
 ('热', 'I-SYMPTOM'),
 ('病', 'O'),
 ('人', 'O'),
 ('不', 'O'),
 ('宜', 'O'),
 ('服', 'O'),
 ('用', 'O'),
 ('。', 'O'),
 (' ', 'O'),
 ('3', 'O'),
 ('.', 'O'),
 ('有', 'O'),
 ('高', 'B-DISEASE'),
 ('血', 'I-DISEASE'),
 ('压', 'I-DISEASE'),
 ('、', 'O'),
 ('心', 'B-DISEASE_GROUP'),
 ('脏', 'I-DISEASE_GROUP'),
 ('病', 'I-DISEASE_GROUP'),
 ('、', 'O'),
 ('肝', 'B-DISEASE_GROUP'),
 ('病', 'I-DISEASE_GROUP'),
 ('、', 'O'),
 ('糖', 'B-DISEASE'),
 ('尿', 'I-DISEASE'),
 ('病', 'I-DISEASE'),
 ('、'

In [19]:
list(sentence)

[]